In [ ]:
import pandas as pd
import numpy as np
import os

# 1)
# Session data usable
## File containing the metadata of the sessions
## Table pointing to the Whisper results

In [ ]:
path_ = '/home/lossioventuraj2/Code/07-Transcripts/'
file_session_usable_ = path_ + 'tables_whisper/Session_data_usable_10_19_2022.csv'
df_ses_use_ = pd.read_csv(file_session_usable_)
df_ses_use_

In [ ]:
df_ses_use_.keys()

# 2)
# Subject DATA

In [ ]:
file_subject = 'tables/Subject_data.xlsx'
df_sub = pd.read_excel(path_ + file_subject)
df_sub

In [ ]:
# Columns of df_sub:
## ID, Study, Therapist_num, site, Female, Age, Tx, CYBOCS_Total_0, CGI_severity_0, Num exposure, files

In [ ]:
df_sub.groupby(['ID','site'])['Tx'].agg(['count']).sort_values(by='count', ascending=False)

In [ ]:
df_sub[['site','Study','Num exposure files']].groupby(['site','Study']).sum()

# 3
# MERGE
# SESSION + SUBJECT
### by ID

#### Since the patient 5012 for the same site 1 (Penn). One used for POTS JR (3) subject and another for a POTS II (2)
#### "We will change the dataframe for 50120

In [ ]:
df_ses_use_['ID_aux'] = df_ses_use_['ID Number']
df_sub['ID_aux'] = df_sub['ID']

### now merge by 
### ID_aux

In [ ]:
df_merge = pd.merge(df_ses_use_,df_sub,how='inner',on='ID_aux')
df_merge

In [ ]:
df_merge_reduced = df_merge[['base_name','site','Study','QC']].groupby(
    ['base_name','site','Study']).aggregate('count').reset_index()
df_merge_reduced.rename(columns={'Study':'study', 'QC': 'exp_session_file'}, inplace=True)
df_merge_reduced

In [ ]:
path_words = '/Volumes/MacDocuments/Projects/7-Transcriptions/TherapyAutoLabeling/words_w_exp_whisper/'

# Words with exposure
# Those that contain 0 and 1
# i.e., files containing at least an exposure

In [ ]:
list_exp_files = os.listdir(path_words)
print(len(list_exp_files))

In [ ]:
list_exp_aux = os.listdir(path_words)
list_exp_files = []
for elem in list_exp_aux:
    if 'whisper' in elem: 
        list_exp_files.append(elem)
print(len(list_exp_files))

In [ ]:
if '.DS_Store' in list_exp_files:
    list_exp_files.remove('.DS_Stores')
print(len(list_exp_files))

In [ ]:
list_data = []
for i in range(len(list_exp_files)):
    df = pd.read_csv(path_words + list_exp_files[i])
    df = df.astype({"Exposure":'int'})
    
    ind = 0
    for index, row in df.iterrows():
        if ind == 0:
            expo = row['Exposure']
            string_concatenate = row['WORD']
        elif ind == df.shape[0]-1:
            if row['Exposure'] == expo:
                string_concatenate = string_concatenate + ' ' + row['WORD']
                list_data.append([expo, string_concatenate, list_exp_files[i]])
            else:
                list_data.append([expo, string_concatenate, list_exp_files[i]])
                
                #Last token
                expo = row['Exposure']
                string_concatenate = row['WORD']
                list_data.append([expo, string_concatenate, list_exp_files[i]])
            
        else:
            if row['Exposure'] == expo:
                string_concatenate = string_concatenate + ' ' + row['WORD']
            else:
                list_data.append([expo, string_concatenate, list_exp_files[i]])
                expo = row['Exposure']
                string_concatenate = row['WORD']
        ind = ind + 1

In [ ]:
result = pd.DataFrame(data=list_data, columns=['Exposure','Text','File'])

In [ ]:
result['Tokens'] = result['Text'].apply(get_number_tokens)
result.reset_index(inplace=True)
result.rename(columns={'index':'ID'},inplace=True)
result

In [ ]:
aux_ = result[['File','Exposure']].groupby(['File']).sum().reset_index()
aux_['base_name'] = aux_['File'].str[:-16] #.str[:-8]
aux_.rename(columns={'Exposure':'exp_wordwexp_file'}, inplace=True)
aux_

## Merge with df_merge (subject + session)

In [ ]:
df_merge_reduced

In [ ]:
df_final = pd.merge(aux_,df_merge_reduced,how='inner',on='base_name')
df_final = df_final[df_final['exp_wordwexp_file']>0.0]
df_final['label'] = df_final['site'].map(str) + df_final['study'].map(str)
df_final['num_files'] = 1
df_final

In [ ]:
print(df_final['exp_session_file'].sum())
print(df_final['exp_wordwexp_file'].sum())

# 4
# Spliting
## in Training, Validation, and Testing

In [ ]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(df_final, test_size=0.322, 
                                               stratify=df_final['label'], random_state=30)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

In [ ]:
validation, testing = train_test_split(testing_data, test_size=0.50, 
                                               stratify=testing_data['label'], random_state=30)

print(f"No. of validation examples: {validation.shape[0]}")
print(f"No. of testing examples: {testing.shape[0]}")

In [ ]:
print(training_data['num_files'].sum())
print(validation['num_files'].sum())
print(testing['num_files'].sum())

In [ ]:
print(training_data['exp_wordwexp_file'].sum())
print(validation['exp_wordwexp_file'].sum())
print(testing['exp_wordwexp_file'].sum())

In [ ]:
df_final[['site','study','num_files']].groupby(['site','study']).sum()

In [ ]:
training_data[['site','study','num_files']].groupby(['site','study']).sum()

In [ ]:
validation[['site','study','num_files']].groupby(['site','study']).sum()

In [ ]:
testing[['site','study','num_files']].groupby(['site','study']).sum()

In [ ]:
df_final.to_pickle('dataset-V1/pkl_1_all.pkl')
training_data.to_pickle('dataset-V1/pkl_2_training.pkl')
validation.to_pickle('dataset-V1/pkl_3_validation.pkl')
testing.to_pickle('dataset-V1/pkl_4_testing.pkl')

### Separate Test from Training, Validation, and Testing

In [ ]:
result

In [ ]:
result.to_excel('dataset-V1/imbalanced_1_all.xlsx', index=None)

In [ ]:
split_df = result[result['File'].isin(training_data['File'].to_list())]
split_df.to_excel('dataset-V1/imbalanced_2_training.xlsx', index=None)
split_df

In [ ]:
split_df = result[result['File'].isin(validation['File'].to_list())]
split_df.to_excel('dataset-V1/imbalanced_3_validation.xlsx', index=None)
split_df

In [ ]:
split_df = result[result['File'].isin(testing['File'].to_list())]
split_df.to_excel('dataset-V1/imbalanced_4_testing.xlsx', index=None)
split_df